In [ ]:
pip install arviz

     |████████████████████████████████| 1.6MB 9.0MB/s 
     |████████████████████████████████| 768kB 39.0MB/s 
     |████████████████████████████████| 4.7MB 38.6MB/s 
     |████████████████████████████████| 317kB 47.1MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [ ]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import pickle
import cmdstanpy
import json
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [ ]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [ ]:
sm_general = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/general_dawid-skene.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b3b91d5d08ea08e32e8380bfec8b2a00 NOW.


In [ ]:
  w = len(np.unique(df['Worker']))
  t = len(np.unique(df['Tasca']))
  a = len(df['Anotació'])
  k = len(np.unique(df['Anotació']))
  t_A = np.array(df['Tasca'])+1
  w_A = np.array(df['Worker'])+1
  ann = np.array(df['Anotació'])+1
  tau_prior = np.array([1,1,1])
  pi_prior = np.ones((k,k))

  dades_sm_gen = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [ ]:
optim = sm_general.optimizing(data=dades_sm_gen)
optim

OrderedDict([('tau', array([0.59040857, 0.29704422, 0.11254721])),
             ('pi', array([[[0.14765821, 0.09129293, 0.76104886],
                      [0.2190873 , 0.58873091, 0.19218179],
                      [0.8501621 , 0.07926495, 0.07057295]],
              
                     [[0.13057424, 0.11073397, 0.75869179],
                      [0.30767198, 0.51725952, 0.17506851],
                      [0.6543249 , 0.12652782, 0.21914728]],
              
                     [[0.07408473, 0.10849105, 0.81742422],
                      [0.21375559, 0.60218531, 0.18405909],
                      [0.62404251, 0.08486799, 0.29108949]],
              
                     [[0.09577377, 0.13825615, 0.76597008],
                      [0.19746324, 0.63425052, 0.16828624],
                      [0.69213084, 0.09325156, 0.2146176 ]],
              
                     [[0.08057932, 0.12261249, 0.79680819],
                      [0.22260584, 0.63068914, 0.14670501],
                      [

In [ ]:
def correct_label_switching(sm,dades_sm,k):
  opt = sm.optimizing(data=dades_sm)

  acumulacio_permutacions=[]
  matrices_pi = opt['pi']
  tau = opt['tau']
  t_C = opt['t_C']
  log_p_t_C = opt['log_p_t_C']
  new_pi = np.zeros((len(matrices_pi),k,k))
  new_tau = np.zeros(k)
  new_t_C = np.zeros((len(t_C),k))
  new_log_p_t_C = np.zeros((len(log_p_t_C),k))

  for w in range(len(matrices_pi)):
    on_hauria_danar = np.argmax(matrices_pi[w],axis=1)
    acumulacio_permutacions.append(on_hauria_danar)
    for i in range(len(matrices_pi[w])): #aquest bucle corregeix les matrius pi
      new_pi[w][on_hauria_danar[i]] = matrices_pi[w][i]
  #amb aquesta part del codi determinem la permutació 'guanyadora' per aplicar-li a tau i t_C
  values, counts = np.unique(acumulacio_permutacions,axis=0, return_counts=True)
  ind = np.argmax(counts)
  permutacio = values[ind]
  #actualitzem tau, t_C i log_P_t_C
  for i in range(len(tau)):
    new_tau[permutacio[i]] = tau[i]

  for t in range(len(opt['t_C'])):
      for i in range(k):
        new_t_C[t][permutacio[i]] = t_C[t][i]
        new_log_p_t_C[t][permutacio[i]] = log_p_t_C[t][i]
  return new_tau,new_pi,new_t_C,new_log_p_t_C,permutacio





In [ ]:
new_tau,new_pi,new_t_C,new_log_p_t_C, permutacio = correct_label_switching(sm_general,dades_sm_gen,k)

In [ ]:
new_tau,new_pi,new_t_C,new_log_p_t_C, permutacio

(array([0.11254858, 0.29706796, 0.59038347]),
 array([[[0.85033674, 0.0790398 , 0.07062346],
         [0.21907949, 0.58874459, 0.19217592],
         [0.14764153, 0.09130061, 0.76105786]],
 
        [[0.65433718, 0.12650747, 0.21915535],
         [0.30763739, 0.51728371, 0.1750789 ],
         [0.13057773, 0.11076129, 0.75866098]],
 
        [[0.62402972, 0.08486919, 0.29110108],
         [0.21375731, 0.60219062, 0.18405207],
         [0.07408899, 0.10850077, 0.81741024]],
 
        [[0.69220328, 0.09319612, 0.2146006 ],
         [0.19745195, 0.63427495, 0.1682731 ],
         [0.09577821, 0.13826044, 0.76596135]],
 
        [[0.55510388, 0.16271194, 0.28218418],
         [0.22262585, 0.63068859, 0.14668556],
         [0.08058579, 0.12264905, 0.79676516]],
 
        [[0.59576088, 0.09815075, 0.30608837],
         [0.20054931, 0.57740835, 0.22204234],
         [0.09019907, 0.09381149, 0.81598944]],
 
        [[0.6433154 , 0.15140301, 0.20528158],
         [0.16931556, 0.60495389, 0.2257305

# Correction of label-switching using an adequated prior

In [ ]:
  w = len(np.unique(df['Worker']))
  t = len(np.unique(df['Tasca']))
  a = len(df['Anotació'])
  k = len(np.unique(df['Anotació']))
  t_A = np.array(df['Tasca'])+1
  w_A = np.array(df['Worker'])+1
  ann = np.array(df['Anotació'])+1
  tau_prior = np.array([1,1,1])
  pi_prior = np.array([[60,20,20],
                       [20,60,20],
                       [20,20,60]])

  dades_sm_gen2 = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [ ]:
optim = sm_general.optimizing(data=dades_sm_gen2)
optim

OrderedDict([('tau', array([0.12325911, 0.27580357, 0.60093733])),
             ('pi', array([[[0.68866325, 0.15665411, 0.15468265],
                      [0.20038533, 0.61167226, 0.1879424 ],
                      [0.16328218, 0.11987177, 0.71684605]],
              
                     [[0.61810316, 0.17931142, 0.20258541],
                      [0.25414819, 0.56647593, 0.17937588],
                      [0.14805665, 0.1322035 , 0.71973984]],
              
                     [[0.61610589, 0.15603146, 0.22786265],
                      [0.19593817, 0.61704951, 0.18701232],
                      [0.10277023, 0.130757  , 0.76647278]],
              
                     [[0.6354491 , 0.166029  , 0.1985219 ],
                      [0.19173246, 0.63004193, 0.17822561],
                      [0.12369436, 0.15493233, 0.72137331]],
              
                     [[0.58747807, 0.1942002 , 0.21832173],
                      [0.20264351, 0.63247961, 0.16487688],
                      [